In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json

scala_version = '2.12'  # your scala version
spark_version = '3.5.0' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:2.8.0' #your kafka version
]
spark = SparkSession.builder.master("local").appName("kafka-example").config("spark.jars.packages", ",".join(packages)).getOrCreate()
spark

In [2]:
packages

['org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0',
 'org.apache.kafka:kafka-clients:2.8.0']

In [3]:
topic_name = 'text1'
kafka_server = 'localhost:9092'

kafkaDf = spark.read.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).option("startingOffsets", "earliest").load()

In [4]:
kafkaDf.show()

+----+--------------------+-----+---------+------+--------------------+-------------+
| key|               value|topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-----+---------+------+--------------------+-------------+
|NULL|[31 39 39 39 2D 3...|text1|        0|     0|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     1|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     2|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     3|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     4|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     5|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     6|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     7|2024-01-20 14:47:...|            0|
|NULL|[31 39 39 39 2D 3...|text1|        0|     8|2024

In [14]:
from time import sleep
from IPython.display import display, clear_output
from pyspark.sql import functions as F
from pyspark.sql.functions import from_json, split, current_date, year, col, when
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

for x in range(0, 2000):
    try:
        df = kafkaDf.selectExpr("CAST(value AS STRING)").select(split("value", ",").alias("csv_values")) \
                        .selectExpr("csv_values[0] as Date", "csv_values[1] as Open", \
                                    "csv_values[2] as High", "csv_values[3] as Low", \
                                    "csv_values[4] as Close", "csv_values[5] as Adj_close", \
                                    "csv_values[6] as Volume")

        df1 = df.alias("copied")
        df1 = df1 \
            .withColumn("Date", col("Date").cast(StringType())) \
            .withColumn("Open", col("Open").cast(DoubleType())) \
            .withColumn("High", col("High").cast(DoubleType())) \
            .withColumn("Low", col("Low").cast(DoubleType())) \
            .withColumn("Close", col("Close").cast(DoubleType())) \
            .withColumn("Adj_close", col("Adj_close").cast(DoubleType())) \
            .withColumn("Volume", col("Volume").cast(IntegerType()))
        df1 = df1.withColumn("Current_date", current_date())

    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")

PySparkTypeError: [NOT_COLUMN] Argument `col` should be a Column, got date.

In [13]:
df1.show()

+----------+-------------------+-------------------+-------------------+-------------------+-------------------+---------+
|      Date|               Open|               High|                Low|              Close|          Adj_close|   Volume|
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+---------+
|1999-01-22|             0.4375| 0.4882810115814209|0.38802099227905273| 0.4101560115814209|  0.376254677772522|271468800|
|1999-01-25|0.44270798563957214|0.45833298563957214| 0.4101560115814209|           0.453125| 0.4156721234321594| 51048000|
|1999-01-26|0.45833298563957214|0.46744799613952637|0.41145798563957214| 0.4179689884185791| 0.3834218978881836| 34320000|
|1999-01-27|0.41927099227905273| 0.4296880066394806|0.39583298563957214|0.41666701436042786|0.38222751021385193| 24436800|
|1999-01-28|0.41666701436042786|0.41927099227905273| 0.4127599895000458| 0.4153650104999542| 0.3810330629348755| 22752000|
|1999-01-29| 0.4

In [11]:
import datetime

cur = datetime.date.today()
print(cur)


2024-01-20
